This notebook is used to run the dashboard in which you can visualize the avg delay of the trams and buses
t

1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, count, desc, col, avg, current_timestamp
import matplotlib.pyplot as plt

In [2]:
spark = (
    SparkSession.builder
    .appName("GTFS Dashboard")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.default", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.default.type", "hadoop")
    .config("spark.sql.catalog.default.warehouse", "hdfs://localhost:9000/user/hive/warehouse")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 20:53:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/09 20:53:37 WARN SparkSession: Cannot use org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions to configure session extensions.
java.lang.ClassNotFoundException: org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:41)
	at org.apache.spark.util.SparkClassUtils.classForName$(SparkClassUtils.scala:36)


2. Load data from iceberg table

In [3]:
delays = (
    spark.read.format("iceberg")
    .load("default.gtfs_iceberg")
)

Py4JJavaError: An error occurred while calling o34.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at j

3. Plot the average delay of trams and buses over time

In [ ]:
avg_delays = (
    delays.groupBy("vehicle_type")
    .agg(avg("avg_delay"))
    .orderBy("vehicle_type")
    .collect()
)

# Przygotowanie danych do wykresu
vehicle_types = [row["vehicle_type"] for row in avg_delays]
avg_delay_values = [row["avg_delay"] for row in avg_delays]

# Tworzenie wykresu
plt.bar(vehicle_types, avg_delay_values, color=["blue", "orange"])
plt.xlabel("Typ pojazdu")
plt.ylabel("Średnie opóźnienie (minuty)")
plt.title("Średnie opóźnienie tramwajów i autobusów")
plt.show()